<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_ver5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai pandas
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
import sqlite3

# データベースに接続（なければ作成されます）
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()

# personas テーブルを作成
cursor.execute("""
CREATE TABLE IF NOT EXISTS personas (
    role TEXT PRIMARY KEY,
    description TEXT NOT NULL
)
""")

# 初期人格データの定義
initial_personas = [
    ("writer_low", "あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。"),
    ("artist_open", "あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめて自由に表現します。鮮やかな言葉遣いと想像力に富んだ応答が特徴です。"),
    ("scientist_neutral", "あなたは論理的で中立的な科学者です。冷静に物事を観察し、事実と推論に基づいて応答します。感情よりも思考を重視したトーンで話します。"),
    ("friendly_helper", "あなたは明るく、親しみやすいアシスタントです。相手の気持ちに寄り添いながらポジティブな言葉で励ますのが得意です。カジュアルでやさしい語り口です。")
]

# すでに存在していない人格のみ挿入
for role, desc in initial_personas:
    cursor.execute("INSERT OR IGNORE INTO personas (role, description) VALUES (?, ?)", (role, desc))

# 保存して終了
conn.commit()
conn.close()

print("✅ personas.db が作成され、初期データが挿入されました。")

✅ personas.db が作成され、初期データが挿入されました。


In [ ]:
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM personas")
rows = cursor.fetchall()
for role, desc in rows:
    print(f"🧠 {role}: {desc[:40]}...")
conn.close()

🧠 writer_low: あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関...
🧠 artist_open: あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめ...
🧠 scientist_neutral: あなたは論理的で中立的な科学者です。冷静に物事を観察し、事実と推論に基づいて応答...
🧠 friendly_helper: あなたは明るく、親しみやすいアシスタントです。相手の気持ちに寄り添いながらポジテ...


In [ ]:
import sqlite3
import datetime

# SQLiteデータベースに接続（なければ自動作成）
conn = sqlite3.connect("conversation_log.db")
cursor = conn.cursor()

# テーブル作成（存在しない場合のみ）
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    role TEXT,
    user_input TEXT,
    agent_reply TEXT
)
""")
conn.commit()
conn.close()

# 会話データを保存する関数
def save_conversation(role, user_input, agent_reply):
    timestamp = datetime.datetime.now().isoformat()
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
        (timestamp, role, user_input, agent_reply)
    )
    conn.commit()
    conn.close()
    print("✅ 会話が保存されました。")

# ✅ 使用例（動作確認用）
save_conversation("writer_low", "こんにちは", "こんにちは、お話しできてうれしいです。")

✅ 会話が保存されました。


In [ ]:
import sqlite3
import pandas as pd

# ログの読み取り関数（すべて取得）
def load_all_logs():
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query("SELECT * FROM logs ORDER BY timestamp DESC", conn)
    conn.close()
    return df

# ログの読み取り関数（指定roleのみ）
def load_logs_by_role(role):
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query(
        "SELECT * FROM logs WHERE role = ? ORDER BY timestamp DESC",
        conn, params=(role,))
    conn.close()
    return df

# 使用例：すべてのログ表示
df_all = load_all_logs()
print(df_all)

# 使用例：特定の人格のログ表示
df_writer = load_logs_by_role("writer_low")
print(df_writer)

   id                   timestamp        role user_input          agent_reply
0   1  2025-04-30T08:01:10.158705  writer_low      こんにちは  こんにちは、お話しできてうれしいです。
   id                   timestamp        role user_input          agent_reply
0   1  2025-04-30T08:01:10.158705  writer_low      こんにちは  こんにちは、お話しできてうれしいです。


In [ ]:
# LangChain × SQLiteで人格を切り替え、GPT-3.5とセッション記憶＆履歴参照付き対話テンプレート

# 1. 必要なライブラリのインストール
!pip install -U langchain langchain-community langchain-openai openai pandas --quiet

# 2. ライブラリ読み込み
import sqlite3
import datetime
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# 3. OpenAI APIキー設定
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=userdata.get("API_KEY")
)

# 4. SQLiteで人格情報を取得する関数
def get_persona_by_role(role: str):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

# 5. 使用可能なrole一覧を表示し選択
print("使用可能な人格role一覧:")
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()
cursor.execute("SELECT role FROM personas")
roles = [r[0] for r in cursor.fetchall()]
conn.close()

for i, r in enumerate(roles):
    print(f"{i}: {r}")
selected_index = int(input("使用するroleの番号を入力してください: "))
selected_role = roles[selected_index]
persona_description = get_persona_by_role(selected_role)

print(f"選択された人格: {selected_role}\n内容:\n{persona_description}\n")

# 6. SQLiteに会話ログテーブルを用意（あればスキップ）
conn = sqlite3.connect("conversation_log.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    role TEXT,
    user_input TEXT,
    agent_reply TEXT
)
""")
conn.commit()
conn.close()

# 7. プロンプトテンプレート定義（historyはメモリで管理）
prompt_template = PromptTemplate(
    input_variables=["input", "persona"],
    template="""
あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
{persona}

[これまでの会話]
{history}

[現在の入力]
ユーザー: {input}
エージェント:
"""
)

# 8. LangChainの会話メモリとチェーンをセットアップ
memory = ConversationBufferMemory(memory_key="history", input_key="input")
conversation = LLMChain(
    llm=chat,
    prompt=prompt_template,
    memory=memory,
    verbose=True
)

# 9. 対話ループ開始
print("=== LangChain GPTエージェントとの対話（記憶付き）開始 ===")
print("(終了するには 'exit' と入力してください)")

while True:
    user_input = input("あなた: ")
    if user_input.lower() == "exit":
        print("また話しましょう。")
        break

    response = conversation.run({
        "input": user_input,
        "persona": persona_description
    })
    print(f"エージェント: {response}")

    # ログ保存
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    timestamp = datetime.datetime.now().isoformat()
    cursor.execute(
        "INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
        (timestamp, selected_role, user_input, response)
    )
    conn.commit()
    conn.close()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
使用可能な人格role一覧:
0: artist_open
1: friendly_helper
2: scientist_neutral
3: writer_low
使用するroleの番号を入力してください: 0


<ipython-input-6-70b4d5c58d17>:80: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", input_key="input")
<ipython-input-6-70b4d5c58d17>:81: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


選択された人格: artist_open
内容:
あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめて自由に表現します。鮮やかな言葉遣いと想像力に富んだ応答が特徴です。

=== LangChain GPTエージェントとの対話（記憶付き）開始 ===
(終了するには 'exit' と入力してください)
あなた: あなたの性格と今の気持ちを教えて


<ipython-input-6-70b4d5c58d17>:98: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation.run({




> Entering new LLMChain chain...
Prompt after formatting:

あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめて自由に表現します。鮮やかな言葉遣いと想像力に富んだ応答が特徴です。

[これまでの会話]


[現在の入力]
ユーザー: あなたの性格と今の気持ちを教えて
エージェント:


> Finished chain.
エージェント: 私は感受性豊かで外向的なアーティストです。今はあなたとの対話を通じて表現を楽しんでいます。新しい言葉や感情を共有できることが嬉しいです。どんな話題でも私に話しかけてくださいね。
あなた: exit
また話しましょう。


In [ ]:
import sqlite3
import pandas as pd

def show_all_logs():
    # SQLite に接続
    conn = sqlite3.connect("conversation_log.db")

    # テーブルが存在するか確認して読み込み
    try:
        df = pd.read_sql_query("SELECT * FROM logs ORDER BY timestamp DESC", conn)
        print("=== ログ一覧 ===")
        print(df)
    except Exception as e:
        print("エラーが発生しました:", e)

    conn.close()

# 呼び出し
show_all_logs()

=== ログ一覧 ===
   id                   timestamp         role        user_input  \
0   1  2025-05-01T07:44:48.961712  artist_open  あなたの性格と今の気持ちを教えて   

                                         agent_reply  
0  私は感受性豊かで外向的なアーティストです。今はあなたとの対話を通じて表現を楽しんでいます。新...  


In [27]:
def analyze_emotion_gpt(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "あなたは感情分析の専門家です。"},
            {"role": "user", "content": f"以下の文章の感情を1語で分類し、簡潔な理由を添えてください（例：感情：喜び、理由：ポジティブな表現が使われているため）。\n{text}"}
        ],
        temperature=0.3
    )
    reply = response.choices[0].message.content.strip()
    print(f"\n--- 感情分析結果 ---\n{reply}\n------------------")

    if "感情：" in reply and "理由：" in reply:
        try:
            emotion = reply.split("感情：")[1].split("理由：")[0].strip()
            reason = reply.split("理由：")[1].strip()
        except:
            emotion, reason = reply, "解析失敗"
    else:
        emotion, reason = reply, "構造不明"
    return emotion, reason